<a href="https://colab.research.google.com/github/jonathas12/python_Ciencia_Dados/blob/main/Atualiza%C3%A7%C3%A3o_Monet%C3%A1ria_IPCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Atualização Monetária - Valores Finais**



In [1]:
import requests
import json

def atualizar_valor_ipca(valor_original, data_inicial_str, data_final_str):
    """
    Atualiza um valor monetário com base na variação do IPCA (série 433)
    consultando a API de Séries Temporais do Banco Central do Brasil.

    Args:
        valor_original (float): O valor a ser atualizado.
        data_inicial_str (str): A data inicial no formato 'MM/AAAA'.
        data_final_str (str): A data final no formato 'MM/AAAA'.

    Returns:
        dict: Um dicionário com os resultados do cálculo, incluindo os dados mensais do IPCA.
    """
    # 1. Converter as datas para o formato que a API do BCB espera (DD/MM/AAAA)
    mes_inicio, ano_inicio = data_inicial_str.split('/')
    mes_fim, ano_fim = data_final_str.split('/')
    data_inicial_api = f"01/{mes_inicio}/{ano_inicio}"
    data_final_api = f"01/{mes_fim}/{ano_fim}"

    # 2. Montar a URL da API para a série do IPCA (código 433)
    url_api = (
        f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.433/dados?"
        f"formato=json&dataInicial={data_inicial_api}&dataFinal={data_final_api}"
    )

    print(f"Buscando dados do IPCA entre {data_inicial_str} e {data_final_str}...")

    try:
        # 3. Fazer a requisição à API
        response = requests.get(url_api)
        response.raise_for_status()  # Lança um erro se a requisição falhar (ex: status 404, 500)
        dados_ipca = response.json()

        if not dados_ipca:
            print("A API não retornou dados para o período selecionado.")
            return None

        # 4. Calcular o fator de correção acumulado
        fator_acumulado = 1.0
        for registro_mensal in dados_ipca:
            variacao_percentual = float(registro_mensal['valor'])
            fator_acumulado *= (1 + variacao_percentual / 100)

        # 5. Calcular o valor final atualizado
        valor_atualizado = valor_original * fator_acumulado

        return {
            "valor_original": valor_original,
            "data_inicial": data_inicial_str,
            "data_final": data_final_str,
            "fator_acumulado": fator_acumulado,
            "inflacao_acumulada_percentual": (fator_acumulado - 1) * 100,
            "valor_atualizado": valor_atualizado,
            "dados_ipca_mensal": dados_ipca # <-- Adicionado aqui
        }

    except requests.exceptions.RequestException as e:
        print(f"Erro ao conectar à API do Banco Central: {e}")
        return None
    except json.JSONDecodeError:
        print("Erro ao processar a resposta da API (não é um JSON válido).")
        return None


# --- DADOS PARA O CÁLCULO ---
valor_para_atualizar = 493.72
data_inicio = "06/2024"
data_fim = "05/2025" # A API pode não ter dados futuros, o ideal é usar a data atual ou passada.

# --- EXECUÇÃO DA FUNÇÃO ---
resultado = atualizar_valor_ipca(valor_para_atualizar, data_inicio, data_fim)

# --- EXIBIÇÃO DO RESULTADO ---
if resultado:
    print("\n--- Resultado da Atualização Monetária ---")
    print(f"Valor Original: R$ {resultado['valor_original']:.2f} em {resultado['data_inicial']}")
    print(f"Valor Atualizado para {resultado['data_final']}: R$ {resultado['valor_atualizado']:.2f}")
    print(f"Fator Acumulado: {resultado['fator_acumulado']:.4f}")
    print(f"Inflação Acumulada no Período: {resultado['inflacao_acumulada_percentual']:.2f}%")

Buscando dados do IPCA entre 06/2024 e 05/2025...

--- Resultado da Atualização Monetária ---
Valor Original: R$ 493.72 em 06/2024
Valor Atualizado para 05/2025: R$ 519.98
Fator Acumulado: 1.0532
Inflação Acumulada no Período: 5.32%


# **2. Atualização Monetária Mês a Mês**



In [3]:
import requests
import pandas as pd
from datetime import datetime

def calcular_tabela_correcao_ipca(valor_original, data_inicial_str, data_final_str):
    """
    Coleta os dados do IPCA (série 433) do Banco Central e cria uma tabela
    detalhada com a correção monetária mês a mês.

    Args:
        valor_original (float): O valor a ser atualizado.
        data_inicial_str (str): A data inicial no formato 'MM/AAAA'.
        data_final_str (str): A data final no formato 'MM/AAAA'.

    Returns:
        pandas.DataFrame: Uma tabela com o detalhamento do cálculo mensal, ou None se falhar.
    """
    # 1. Formatar as datas para a consulta na API
    mes_inicio, ano_inicio = data_inicial_str.split('/')
    mes_fim, ano_fim = data_final_str.split('/')
    data_inicial_api = f"01/{mes_inicio}/{ano_inicio}"
    data_final_api = f"01/{mes_fim}/{ano_fim}"

    # 2. Montar a URL da API para a série do IPCA (código 433)
    url_api = (
        f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.433/dados?"
        f"formato=json&dataInicial={data_inicial_api}&dataFinal={data_final_api}"
    )

    print(f"Buscando dados do IPCA entre {data_inicial_str} e {data_final_str}...")

    try:
        # 3. Fazer a requisição à API e obter os dados
        response = requests.get(url_api, timeout=10) # Adicionado timeout
        response.raise_for_status()
        dados_ipca = response.json()

        if not dados_ipca:
            print("Nenhum dado encontrado para o período especificado.")
            return None

        # 4. Processar os dados e construir a tabela
        historico_calculo = []
        valor_acumulado = valor_original

        for registro in dados_ipca:
            data_mes = datetime.strptime(registro['data'], '%d/%m/%Y').strftime('%m/%Y')
            variacao_percentual = float(registro['valor'])
            fator_mensal = 1 + (variacao_percentual / 100)

            valor_inicio_mes = valor_acumulado
            valor_fim_mes = valor_acumulado * fator_mensal

            historico_calculo.append({
                'Mês/Ano': data_mes,
                'IPCA (%)': variacao_percentual,
                'Valor no Início do Mês': valor_inicio_mes,
                'Valor Corrigido no Fim do Mês': valor_fim_mes
            })

            valor_acumulado = valor_fim_mes

        # 5. Criar um DataFrame (tabela) com os resultados
        tabela_resultado = pd.DataFrame(historico_calculo)
        return tabela_resultado

    except requests.exceptions.RequestException as e:
        print(f"ERRO: Não foi possível conectar à API do Banco Central. Verifique sua conexão. Detalhe: {e}")
        return None

# --- DADOS PARA O CÁLCULO (você pode alterar estes valores) ---
valor_para_atualizar = 493.72
data_inicio = "06/2024"
# Usando a data mais recente disponível para garantir que os dados já foram publicados
data_fim = "05/2025"

# --- EXECUÇÃO E EXIBIÇÃO DA TABELA ---
tabela_correcao = calcular_tabela_correcao_ipca(valor_para_atualizar, data_inicio, data_fim)

if tabela_correcao is not None:
    print("\n--- Tabela de Correção Monetária Detalhada (Mês a Mês) ---")

    # Configurar o pandas para formatar os valores monetários na exibição
    pd.options.display.float_format = 'R$ {:,.2f}'.format

    # O método to_string garante que a tabela seja impressa por completo
    print(tabela_correcao.to_string(index=False))

    # --- EXIBIÇÃO DO RESUMO FINAL ---
    valor_final = tabela_correcao['Valor Corrigido no Fim do Mês'].iloc[-1]
    inflacao_total = (valor_final / valor_para_atualizar - 1) * 100

    print("\n--- Resumo do Período ---")
    print(f"Valor Original em {data_inicio}: R$ {valor_para_atualizar:,.2f}")
    print(f"Valor Final Corrigido para {data_fim}: R$ {valor_final:,.2f}")
    print(f"Inflação Acumulada no Período: {inflacao_total:.2f}%")
    print("-------------------------")

Buscando dados do IPCA entre 06/2024 e 05/2025...

--- Tabela de Correção Monetária Detalhada (Mês a Mês) ---
Mês/Ano  IPCA (%)  Valor no Início do Mês  Valor Corrigido no Fim do Mês
06/2024   R$ 0.21               R$ 493.72                      R$ 494.76
07/2024   R$ 0.38               R$ 494.76                      R$ 496.64
08/2024  R$ -0.02               R$ 496.64                      R$ 496.54
09/2024   R$ 0.44               R$ 496.54                      R$ 498.72
10/2024   R$ 0.56               R$ 498.72                      R$ 501.52
11/2024   R$ 0.39               R$ 501.52                      R$ 503.47
12/2024   R$ 0.52               R$ 503.47                      R$ 506.09
01/2025   R$ 0.16               R$ 506.09                      R$ 506.90
02/2025   R$ 1.31               R$ 506.90                      R$ 513.54
03/2025   R$ 0.56               R$ 513.54                      R$ 516.42
04/2025   R$ 0.43               R$ 516.42                      R$ 518.64
05/2025   R$ 0

In [ ]:
import sys

# Defina o nome do arquivo que deseja criar
nome_do_arquivo = 'relatorio_correcao_ipca.xlsx'

if tabela_correcao is not None and not tabela_correcao.empty:
    print("\n--- Gerando arquivo Excel ---")

    # Cria uma cópia para formatar antes de salvar, preservando o original
    tabela_para_salvar = tabela_correcao.copy()

    # Formata as colunas de valor como texto com 2 casas decimais
    # Isso melhora a visualização diretamente no Excel
    tabela_para_salvar['Valor no Início do Mês'] = tabela_para_salvar['Valor no Início do Mês'].map('{:.2f}'.format)
    tabela_para_salvar['Valor Corrigido no Fim do Mês'] = tabela_para_salvar['Valor Corrigido no Fim do Mês'].map('{:.2f}'.format)

    # Verifica se o código está rodando no Google Colab
    if 'google.colab' in sys.modules:
        # Se estiver no COLAB, usa a biblioteca 'files' para forçar o download
        from google.colab import files

        print(f"Ambiente Google Colab detectado. Iniciando download do arquivo '{nome_do_arquivo}'...")
        tabela_para_salvar.to_excel(nome_do_arquivo, index=False, engine='openpyxl')
        files.download(nome_do_arquivo)

    else:
        # Se estiver em um ambiente local (Jupyter, etc.), salva o arquivo na mesma pasta do script
        try:
            print(f"Ambiente local detectado. Salvando o arquivo '{nome_do_arquivo}'...")
            tabela_para_salvar.to_excel(nome_do_arquivo, index=False, engine='openpyxl')
            print(f"\nArquivo salvo com sucesso na mesma pasta onde este script está sendo executado.")
        except Exception as e:
            print(f"\nOcorreu um erro ao salvar o arquivo: {e}")
            print("Verifique se você tem permissão de escrita na pasta ou se o arquivo já está aberto.")

else:
    print("\nO DataFrame está vazio ou não foi criado, portanto o arquivo Excel não foi gerado.")


--- Gerando arquivo Excel ---
Ambiente Google Colab detectado. Iniciando download do arquivo 'relatorio_correcao_ipca.xlsx'...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>